# Домашняя работа по теме "Машинное обучение ранжированию"

В этом ДЗ мы:
- научимся работать со стандартным датасетом для машинного обучения ранжированию [MSLR](https://www.microsoft.com/en-us/research/project/mslr/)
- попробуем применить на практике все то, чему мы научились на семинаре

## Как будет происходить сдача ДЗ

Вам надо:
- форкнуть эту репу
- создать бранч в котором вы дальше будете работать
- реализовать класс Model в этом ноутбуке
- убедиться, что ваша реализация выбивает NDCG@10 выше бейзлайна (см. ниже)
- запушить ваш бранч и поставить Pull Request
- в комментарии написать какой скор вы выбили

В таком случае мы (организаторы):

- счекаутим вашу бранчу
- проверим что ваша реализация действительно выбивает заявленный скор

Предполагается, что и вы, и мы работаем в виртаульном окружении как в семинаре про машинное обучение ранжированию: seminars/7-learning-to-rank/requirements.txt(подробнее про работу с виртуальными окружениями README в корне этой репы).

Оценка:
- За выбитый скор больше **0.507** назначаем **5** баллов, за скор больше (или равно) **0.510** назначаем максимальный балл -- 10 баллов
- Тот из участников кто выбъет самый высокий скор получит еще +10 баллов

При сдаче кода важно помнить о том, что:
- В коде не должно быть захардкоженных с потолка взятых гиперпараметров (таких как число деревьев, learning rate и т.п.) -- обязательно должен быть представлен код который их подбирает!
- Решение должно быть стабильно от запуска к запуску (на CPU) т.е. все seed'ы для генераторов случайных чисел должны быть фиксированы
- Мы (организаторы) будем запускать код на CPU поэтому, даже если вы использовали для подбора параметров GPU, финальный скор надо репортить на CPU

## Пререквизиты

Импортируем все что нам понадобится для дальнейшей работы:

In [1]:
import pathlib
from timeit import default_timer as timer

import numpy as np
import pandas as pd

from catboost import datasets, utils

## Датасет MSLR (Microsoft Learning to Rank)

Загрузим датасет MSLR.

Полный датасет можно скачать с официального сайта: https://www.microsoft.com/en-us/research/project/mslr/

Строго говоря, он состоит их 2х частей:

- основной датасет MSLR-WEB30K -- он содержит более 30 тыс. запросов
- "маленький" датасет MSLR-WEB10K, который содержит только 10 тыс. запросов и является случайным сэмплом датасета MSLR-WEB30K

в этом ДЗ мы будем работать с MSLR-WEB10K, т.к. полная версия датасета может просто не поместиться у нас в RAM (и, тем более, в память видеокарты если мы учимся на GPU)

Будем считать, что мы самостоятельно скачали датасет MSLR-WEB10K с официального сайта, поместили его в папку КОРЕНЬ-ЭТОЙ-РЕПЫ/data/mslr-web10k и раззиповали.

В результате у нас должна получиться следующая структура папок:

In [2]:
!ls ../../data

"ls" �� ���� ����७��� ��� ���譥�
��������, �ᯮ��塞�� �ணࠬ��� ��� ������ 䠩���.


In [3]:
!ls -lh ../../data/mslr-web10k/

"ls" �� ���� ����७��� ��� ���譥�
��������, �ᯮ��塞�� �ணࠬ��� ��� ������ 䠩���.


Заметим, что датасет довольно большой, в распакованном виде он весит 7.7 GB.

Датасет состоит из нескольких фолдов, которые по сути представляют из себя разные разбиения одних и тех же данных на обучающее, валидационное и тестовые множеста.

Дальше мы будем использовать только первый фолд: Fold1.

Заглянем внутрь:

In [5]:
!ls -lh ../../data/mslr-web10k/Fold1

"ls" �� ���� ����७��� ��� ���譥�
��������, �ᯮ��塞�� �ணࠬ��� ��� ������ 䠩���.


Видим, что у нас 3 файла с говорящими названиями, соответсвующими сплитам нашего датасета.

Посмотрим на содержимое одного из файлов:

In [6]:
!head -n 1 ../../data/mslr-web10k/Fold1/train.txt

"head" �� ���� ����७��� ��� ���譥�
��������, �ᯮ��塞�� �ணࠬ��� ��� ������ 䠩���.


Видим, что данные лежат в уже знакомом нам по семинару формате:

- В первой колонке лежит таргет (оценка асессора), по 5-балльной шкале релевантности: от 0 до 4 (включительно)
- Во второй колонке лежит ID запроса, по которому можно сгруппировать все оценки документов в рамках одного и того же запроса
- Дальше идет вектор из 128 фичей (таких как значения BM25 и т.п.), их точная природа нам сейчас на важна

В файле qid и все-фичи кодируются в формате КЛЮЧ:ЗНАЧЕНИЕ, напр. 130:116 -- тут 130 это номер фичи, а 116 -- ее значение.

Такой формат в мире машинного обучения часто называют svm light формат (в честь когда-то популярной библиотеки SVM-Light)

Напишем немного вспомогательного кода для загрузки этого датасета:

In [4]:
def generate_column_names(num_features):
    """Generates column names for LETOR-like datasets"""
    columns = ['label', 'qid']
    for i in range(num_features):
        column = f"feature_{i+1}"
        columns.append(column)
    return columns
    
def load_svmlight_file(input_file, max_num_lines=0):
    """Loads dataset split in SVM-Light format"""
    def _parse_field(field):
        parts = field.split(':')
        if len(parts) != 2:
            raise Exception(f"invalid number of parts in field {field}")
        return parts

    num_features = 136
    exp_num_fields = num_features + 2
    num_lines = 0
    X = []
    with open(input_file, 'rt') as f:
        for line in f:
            try:
                num_lines += 1
                                  
                # Parse into fields
                fields = line.rstrip().split(' ')
                num_fields = len(fields)
                if num_fields != exp_num_fields:
                    raise Exception(f"invalid number of fields {num_fields}")
    
                # Parse every field
                x = np.zeros(exp_num_fields, dtype=np.float32)
                label = int(fields[0])
                x[0] = label
                _, qid_str = _parse_field(fields[1])
                qid = int(qid_str)
                x[1] = qid
                for i, field in enumerate(fields[2:]):
                    _, feature_str = _parse_field(field)
                    x[i+2] = float(feature_str)
    
                # Add new object
                X.append(x)
                if num_lines % 50000 == 0:
                    print(f"Loaded {num_lines} lines...")
                if max_num_lines > 0 and num_lines == max_num_lines:
                    print(f"WARNING: stop loading, line limit reached: max_num_lines = {max_num_lines} input_file = {input_file}")
                    break
            except Exception as e:
                raise Exception(f"error at line {num_lines} in {input_file}") from e
    
    # To pandas
    df = pd.DataFrame(X, columns=generate_column_names(num_features))
    print(f"Loaded SVM-Light file {input_file}")
    return df

И теперь загрузим датасет:

In [5]:
fold_dir = pathlib.Path("Fold1")

df_train = load_svmlight_file(fold_dir.joinpath("train.txt"))
df_valid = load_svmlight_file(fold_dir.joinpath("vali.txt"))
df_test = load_svmlight_file(fold_dir.joinpath("test.txt"))
print(f"Dataset loaded from fold_dir {fold_dir}")

Loaded 50000 lines...
Loaded 100000 lines...
Loaded 150000 lines...
Loaded 200000 lines...
Loaded 250000 lines...
Loaded 300000 lines...
Loaded 350000 lines...
Loaded 400000 lines...
Loaded 450000 lines...
Loaded 500000 lines...
Loaded 550000 lines...
Loaded 600000 lines...
Loaded 650000 lines...
Loaded 700000 lines...
Loaded SVM-Light file Fold1\train.txt
Loaded 50000 lines...
Loaded 100000 lines...
Loaded 150000 lines...
Loaded 200000 lines...
Loaded SVM-Light file Fold1\vali.txt
Loaded 50000 lines...
Loaded 100000 lines...
Loaded 150000 lines...
Loaded 200000 lines...
Loaded SVM-Light file Fold1\test.txt
Dataset loaded from fold_dir Fold1


Посмотрим на данные:

In [9]:
print(df_train.head(5))

   label  qid  feature_1  feature_2  feature_3  feature_4  feature_5  \
0    2.0  1.0        3.0        3.0        0.0        0.0        3.0   
1    2.0  1.0        3.0        0.0        3.0        0.0        3.0   
2    0.0  1.0        3.0        0.0        2.0        0.0        3.0   
3    2.0  1.0        3.0        0.0        3.0        0.0        3.0   
4    1.0  1.0        3.0        0.0        3.0        0.0        3.0   

   feature_6  feature_7  feature_8  ...  feature_127  feature_128  \
0        1.0        1.0   0.000000  ...         62.0   11089534.0   
1        1.0        0.0   1.000000  ...         54.0   11089534.0   
2        1.0        0.0   0.666667  ...         45.0          3.0   
3        1.0        0.0   1.000000  ...         56.0   11089534.0   
4        1.0        0.0   1.000000  ...         64.0          5.0   

   feature_129  feature_130  feature_131  feature_132  feature_133  \
0          2.0        116.0      64034.0         13.0          3.0   
1          2

Т.е. теперь мы видим что данные доступны в точно таком же виде, как это было в семинаре.

Проведем небольшой EDA.

Всего у нас 723 тыс. документов в трейне:

In [10]:
print(df_train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 723412 entries, 0 to 723411
Columns: 138 entries, label to feature_136
dtypes: float32(138)
memory usage: 380.8 MB
None


235 тыс. документов в валидации:

In [11]:
print(df_valid.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 235259 entries, 0 to 235258
Columns: 138 entries, label to feature_136
dtypes: float32(138)
memory usage: 123.8 MB
None


И 241 тыс. документов в тесте:

In [12]:
print(df_test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 241521 entries, 0 to 241520
Columns: 138 entries, label to feature_136
dtypes: float32(138)
memory usage: 127.1 MB
None


Сколько у нас запросов?

In [6]:
num_queries_train = df_train['qid'].nunique()
num_queries_valid = df_valid['qid'].nunique()
num_queries_test = df_test['qid'].nunique()
print(f"Got {num_queries_train} train, {num_queries_valid} valid and {num_queries_test} test queries")

Got 6000 train, 2000 valid and 2000 test queries


## Обучаем модель

Теперь можно приступить непосредственно к обучению модели. 

Объявим класс модели, который надо будем заимлементить в этом ДЗ:

In [7]:
def normalize_(vector: pd.Series) -> np.ndarray:
    max_relevance = np.max(vector)
    return (vector / max_relevance).values


In [8]:
y_train = normalize_(df_train['label'])
y_valid = normalize_(df_valid['label'])
df_train_proc = df_train.drop(['label'], axis=1)
df_valid_proc = df_valid.drop(['label'], axis=1)

In [16]:
!pip install hyperopt==0.2.5

In [50]:
from catboost import CatBoostRanker, Pool, utils
import hyperopt
from numpy.random import RandomState

class Model:
    def __init__(self):
        self.best = None
    
    def hyperopt_objective(self, params):
        model = CatBoostRanker(
            **params,
            random_seed=42,
            verbose=False,
            task_type='GPU',
            border_count=127           
        )
        model.fit(self.train, eval_set=self.valid)
        score = utils.eval_metric(self.valid.get_label(), model.predict(self.valid), 
                                  'NDCG:top=10;type=Exp', group_id=self.valid.get_group_id_hash())[0]
        
        return 1 - score # as hyperopt minimises

    @staticmethod
    def normalize_(vector: pd.Series) -> pd.Series:
        max_relevance = np.max(vector)
        return vector / max_relevance

    def params_optimize(self, df_train, df_valid):
        y_train = Model.normalize_(df_train['label'])
        y_valid = Model.normalize_(df_valid['label'])
        df_train_proc = df_train.drop(['label', 'qid'], axis=1)
        df_valid_proc = df_valid.drop(['label', 'qid'], axis=1)

        queries_train = df_train['qid'].astype('Int32')
        queries_test = df_valid['qid'].astype('Int32')

        train = Pool(
            data=df_train_proc,
            label=y_train,
            group_id=queries_train
        )

        test = Pool(
            data=df_valid_proc,
            label=y_valid,
            group_id=queries_test
        )

        self.train = train
        self.valid = test

        params_space = {
            'learning_rate': hyperopt.hp.uniform('learning_rate', 0.1, 0.3),
            'depth': hyperopt.hp.quniform('depth', 4, 10, 1),
            'l2_leaf_reg': hyperopt.hp.uniform('l2_leaf_reg', 1, 20),
            'bagging_temperature': hyperopt.hp.uniform('bagging_temperature', 0.8, 1.2),
            'eval_metric': 'NDCG:top=10;type=Exp',
            'loss_function': 'YetiRank',
            'grow_policy': 'Depthwise',
            'min_data_in_leaf': hyperopt.hp.quniform('min_data_in_leaf', 50, 100, 1)
        }

        trials = hyperopt.Trials()
        
        best = hyperopt.fmin(
            self.hyperopt_objective,
            space=params_space,
            algo=hyperopt.rand.suggest,
            max_evals=200,
            trials=trials,
            rstate=RandomState(42)            
        )
        self.best = best
        return best

    def fit(self, df_train, df_valid, device='CPU'):
        y_train = Model.normalize_(df_train['label'])
        y_valid = Model.normalize_(df_valid['label'])
        df_train_proc = df_train.drop(['label', 'qid'], axis=1)
        df_valid_proc = df_valid.drop(['label', 'qid'], axis=1)

        queries_train = df_train['qid'].astype('Int32')
        queries_test = df_valid['qid'].astype('Int32')

        train = Pool(
            data=df_train_proc,
            label=y_train,
            group_id=queries_train,
        )

        test = Pool(
            data=df_valid_proc,
            label=y_valid,
            group_id=queries_test
        )

        self.train = train
        self.valid = test

        if self.best:
            self.model = CatBoostRanker(
                **self.best,
                eval_metric='NDCG:top=10;type=Exp',
                random_seed=42,
                verbose=False,
                loss_function='YetiRank',
                iterations=15000, #берём с запасом
                use_best_model=True,
                task_type=device,
                border_count=127, #фиксируем стандартное для CPU значение
            )
        else:
            print('Используем параметры по умолчанию')
            self.model = CatBoostRanker(
                eval_metric='NDCG:top=10;type=Exp',
                random_seed=42,
                verbose=False,
                loss_function='YetiRank',
                iterations=15000, #берём с запасом
                use_best_model=True
            )

        self.model.fit(train, eval_set=test, plot=True)


    def predict(self, df_test):
        return self.model.predict(df_test)

Подберем параметры:

In [59]:
# Create model
model = Model()

opt_params = model.params_optimize(df_train, df_valid)
opt_params

  0%|          | 0/200 [00:00<?, ?trial/s, best loss=?]

Default metric period is 5 because PFound
, NDCG
 is/are not implemented for GPU

Metric PFound is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time

Metric NDCG:top=10;type=Exp is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time



  0%|          | 1/200 [01:05<3:38:07, 65.77s/trial, best loss: 0.42975035375932336]

Default metric period is 5 because PFound
, NDCG
 is/are not implemented for GPU

Metric PFound is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time

Metric NDCG:top=10;type=Exp is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time



  1%|          | 2/200 [02:11<3:37:05, 65.79s/trial, best loss: 0.42975035375932336]

...

Default metric period is 5 because PFound
, NDCG
 is/are not implemented for GPU

Metric PFound is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time

Metric NDCG:top=10;type=Exp is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time



100%|█████████▉| 199/200 [4:12:24<01:13, 73.56s/trial, best loss: 0.42678748280825274]

Default metric period is 5 because PFound
, NDCG
 is/are not implemented for GPU

Metric PFound is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time

Metric NDCG:top=10;type=Exp is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time



100%|██████████| 200/200 [4:13:54<00:00, 76.17s/trial, best loss: 0.42678748280825274]


{'bagging_temperature': 0.8118653840863571,
 'depth': 9.0,
 'l2_leaf_reg': 2.0359699937555145,
 'learning_rate': 0.2247588142495665,
 'min_data_in_leaf': 74.0}

Таким образом, имеем следующий набор параметров:
```python
model.best = {'bagging_temperature': 0.8118653840863571,
 'depth': 9.0,
 'l2_leaf_reg': 2.0359699937555145,
 'learning_rate': 0.2247588142495665,
 'min_data_in_leaf': 74.0}
 ```

Применим модель:

In [51]:
model = Model()
model.best = {'bagging_temperature': 0.8118653840863571,
 'depth': 9.0,
 'l2_leaf_reg': 2.0359699937555145,
 'learning_rate': 0.2247588142495665,
 'min_data_in_leaf': 74.0}


# Fit
start = timer()
model.fit(df_train, df_valid, device='GPU')
elapsed = timer() - start
print(f"Model fit: elapsed = {elapsed:.3f}")

# Predict
y_hat_test = model.predict(df_test)
print(f"Predicted: y_hat_test.shape = {y_hat_test.shape}")

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Default metric period is 5 because PFound, NDCG is/are not implemented for GPU
Metric PFound is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric NDCG:top=10;type=Exp is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


Model fit: elapsed = 1205.187
Predicted: y_hat_test.shape = (241521,)


Теперь, имея предикты, можно посчитать метрики качества:

In [54]:
def compute_metrics(y_true, y_hat, q):
    # List of metrics to evaluate
    eval_metrics = ['NDCG:top=10;type=Exp']
    
    for eval_metric in eval_metrics:
        scores = utils.eval_metric(y_true, y_hat, eval_metric, group_id=q)
    
        # Print scores
        print(f"metric = {eval_metric} score = {scores[0]:.3f}")

# Get test targets and groups
y_test = df_test['label'].to_numpy()
q_test = df_test['qid'].to_numpy().astype('uint32')
    
# Compute metrics on test
compute_metrics(y_test, y_hat_test, q_test)

metric = NDCG:top=10;type=Exp score = 0.512


Ожидаем, что ваша модель покажет результаты выше бейзлайна!

In [55]:
model.model.save_model('final_model.cb')